In [1]:
!pip install datasets

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from datasets import Dataset
from transformers.pipelines.pt_utils import KeyDataset
from tqdm import tqdm

In [3]:
torch.random.manual_seed(0)
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-0.5B-Instruct",
    device_map="cuda",
    #device_map='cpu',
    torch_dtype="auto",
    trust_remote_code=True,
)

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [4]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-0.5B-Instruct", padding_side='left')

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [5]:
import pandas as pd

df = pd.read_json("hf://datasets/MuskumPillerum/General-Knowledge/output.json")

df['n_words'] = df.apply(lambda x: len(str(x['Answer']).split(' ')), axis=1)

df = df[df['n_words'] <= 300]

In [26]:
prompt = "Name some festivals celebrated in India?"
message = lambda q: [
    {"role": "system", "content": "You are a helpful assistant. Please, make your answers of moderate length"},
    {"role": "user", "content": q}
]
text = lambda msg: tokenizer.apply_chat_template(
    msg,
    tokenize=False,
    add_generation_prompt=True
)
text

<function __main__.<lambda>(msg)>

In [36]:
batch_size = 256 
pipe = pipeline(
    "text-generation",
    #"question-answering",
    model=model,
    tokenizer=tokenizer,
    batch_size=batch_size
)

generation_args = {
    "max_new_tokens": 250, # chosen by rule-of-thumb 3/4 words per token
    "return_full_text": True,
    "temperature": 0.2,
    "do_sample": True,
}
i_check = 0

In [21]:
print(pipe(text(message("Describe the different stages in the software development lifecycle.?")), **generation_args)[0]['generated_text'])

<|im_start|>system
You are a helpful assistant. Please, make your answers of moderate length<|im_end|>
<|im_start|>user
Describe the different stages in the software development lifecycle.?<|im_end|>
<|im_start|>assistant
The Software Development Lifecycle (SDL) is a structured approach to software development that outlines the various phases and activities involved in creating, testing, and maintaining software products. This process can be broken down into several key stages:

1. **Requirements Gathering**: At the beginning of the SDL, developers gather requirements from stakeholders such as users, customers, or project managers. These requirements define what the software should do and how it will work.

2. **Analysis**: Once requirements have been gathered, analysts analyze these requirements to understand their complexity, dependencies, and potential impact on other parts of the system. This phase involves identifying the problem domain, defining the scope, and understanding the b

In [37]:
df_qwen25_answers = pd.DataFrame(columns=['Questions', 'Answers'])

In [34]:
df1 = df.iloc[i_check:, :]
dataset = Dataset.from_dict({
    'index': list(range(0, df1.shape[0])),

    'Question': [text(message(row['Question'])) for i, row in df1.iterrows()]
})

In [31]:
dataset[0]

{'index': 0,
 'Question': '<|im_start|>system\nYou are a helpful assistant. Please, make your answers of moderate length<|im_end|>\n<|im_start|>user\nWhat is Artificial Intelligence?<|im_end|>\n<|im_start|>assistant\n'}

In [38]:
from tqdm import tqdm

In [39]:
res = []
i=0
freq_to_save = 1000
i_update = 0
for out in tqdm(pipe(KeyDataset(dataset, 'Question'), **generation_args),
                total=df1.shape[0]):
  #print(out)
    out1 = out[0]['generated_text'].split('<|im_end|>') # роль, вопрос и ответ

    q = out1[1][qs:]
    answ = out1[2][answs+1:]
    #print(f"{q} ||| {answ}")
    
    df_qwen25_answers.loc[i, :] = [q, answ]
    i+=1
    i_update+=1
    if i_update > freq_to_save or i+1 == df1.shape[0]: #чекпоинт сохранения или конец обработки
        i_update = 0
        df_qwen25_answers.to_csv('qwen25_answers0.csv')


100%|██████████| 37555/37555 [3:06:09<00:00,  3.36it/s]  


In [ ]:
step = 1
freq_to_save = 10
j = 0
cnt = 0
i_check = 1110+824+1549+424+639+429+2469+2669 #проверить последнюю генерацию _5
for i in tqdm(range(i_check, df.shape[0], step)):
    rows = df.iloc[i:i+step, :]
    #q = row['Question']
    messages = [
    {"role": "system", "content": "You are a helpful AI assistant."}
    ] + [{"role": "user", "content": row['Question']} for i, row in rows.iterrows()]

    questions = '<>'.join([row['Question'] for i, row in rows.iterrows()])

    #print(messages)
    #break
    answers = pipe(messages, **generation_args)[0]['generated_text']
    #print(len(answer.split('\n\n\n')))
    #break
    df_qwen25_answers.loc[j, :] = [questions, answers]
    j+=1
    cnt+=1

    if cnt == freq_to_save  or i+step > df.shape[0]:
        cnt = 0
        df_qwen25_answers.to_csv('phi_answers6.csv')
    #чекпоинт 1110 (индекс 1109)

  9%|▉         | 2672/30111 [10:01:53<132:03:52, 17.33s/it]

In [40]:
df_qwen25_answers.to_csv('qwen25_answers0.csv')